# 第8回講義 演習

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 目次

課題1. 計算グラフ上での系列走査

課題2. Recurrent Neural Network (RNN) によるIMDbのsentiment analysis
1. データセットの読み込み
2. 各層クラスの実装
3. 計算グラフ構築 & パラメータの更新設定
4. 学習

課題3. Cellを用いたRNNの記述

課題4. Long short-term memory (LSTM)

【補足】Gradient Clipping（長系列への対処法）

【補足】 Eager Executionについて

## 課題1. 計算グラフ上での系列走査

tensorflowの計算グラフ上でRNNで扱うような系列全体を走査するには、 `tf.scan`関数を使用します。

系列全体にある関数を適用しながらfor文を回すことに対応します。

参考:
https://www.tensorflow.org/api_docs/python/tf/scan

#### 例:Accumulation function for vector

In [2]:
x = tf.placeholder(tf.float32)

def fn(a, x):
    return a + x

res = tf.scan(fn=fn, elems=x)

In [3]:
with tf.Session() as sess:
    print(sess.run(res, feed_dict={x: np.array([1, 2, 3, 4, 5, 6])}))

[ 1.  3.  6. 10. 15. 21.]


このように`tf.scan`関数では、 2つの引数を取る関数`fn`と系列`elems`を指定します。

`fn`の引数はそれぞれ役割が以下のように決まっています。
  - 第1引数: 前ステップのfnの出力
  - 第2引数: 今ステップの入力(elems)
  
つまり、`tf.scan`は以下のように`elems`の各要素に`fn`を適用します。(N：elemsの系列長)

$f_0 = elems[0]$

$f_1={\rm fn}(f_0, elems[1])$

$f_2={\rm fn}(f_1, elems[2])$

$\vdots$

$f_{N-1}={\rm fn}(f_{N-2}, elems[N-1])$

`tf.scan`の出力はこの$f_0, \ldots, f_{N-1}$の系列です。

先程の例では

$f_0 = 1$

$f_1={\rm fn}(f_0, elems[1]) = 1 + 2 = 3$

$f_2={\rm fn}(f_1, elems[2]) = 3 + 3 = 6$

$\vdots$

となり、最終的に`tf.scan`の出力は$[1, 3, 6, 10, 15, 21]$となったわけです。

#### 例:Accumulation function for matrix

In [4]:
x = tf.placeholder(tf.float32)

def fn(a, x):
    return a + x

res = tf.scan(fn=fn, elems=x)

In [5]:
with tf.Session() as sess:
    print(sess.run(res, feed_dict={
            x: np.array([[1, 2, 3, 4, 5],
                         [1, 2, 3, 4, 5],
                         [1, 2, 3, 4, 5]])
    }))

[[ 1.  2.  3.  4.  5.]
 [ 2.  4.  6.  8. 10.]
 [ 3.  6.  9. 12. 15.]]


#### 例: initializer

`tf.scan`関数にはinitializer引数を指定することも可能です。

これによりloopの初期値を明示的に指定でき、以下のように機能します。

$f_0={\rm fn}(initializer, elems[0])$

$f_1={\rm fn}(f_1, elems[1])$

$f_2={\rm fn}(f_2, elems[2])$

$\vdots$

$f_{N-1}={\rm fn}(f_{N-1}, elems[N-1])$

なおinitializerがない場合、上記のように入力系列の最初が初期値となります。

In [6]:
x = (tf.placeholder(tf.float32), tf.placeholder(tf.float32))
init = tf.placeholder(tf.float32)

def fn(a, x):
    return x[0] - x[1] + a

res = tf.scan(fn=fn, elems=x, initializer=init)

In [7]:
elems = np.array([1, 2, 3, 4, 5, 6])
with tf.Session() as sess:
    print(sess.run(res, feed_dict={
            x: (elems+1, elems),
            init: np.array(0)
    }))

[1. 2. 3. 4. 5. 6.]


#### 例: フィボナッチ数列（initializerを利用）
$F_0 = 0,$
$F_1 = 1,$
$F_{n + 2} = F_n + F_{n + 1} (n ≧ 0)$

In [8]:
x = tf.placeholder(tf.float32)
init = (tf.placeholder(tf.float32), tf.placeholder(tf.float32))

def fn(a, _):
    return (a[1], a[0]+a[1])

res = tf.scan(fn=fn, elems=x, initializer=init)

# fibonaccis == ([1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13])

In [9]:
with tf.Session() as sess:
    print(sess.run(res, feed_dict={
            x: np.array([0, 0, 0, 0, 0, 0]),
            init: (np.array(0), np.array(1))
    }))

(array([1., 1., 2., 3., 5., 8.], dtype=float32), array([ 1.,  2.,  3.,  5.,  8., 13.], dtype=float32))


## 課題2. Recurrent Neural Network (RNN) によるIMDbのsentiment analysis

IMDb (Internet Movie Database) と呼ばれるデータセットには、映画のレビュー文とその評価がpositiveかnegativeかが記録されています。

<div style="text-align: center;">【データセットのイメージ】</div>

| レビュー | 評価 |
|:--------:|:-------------:|
|Where's Michael Caine when you need him? I've ...|negative|
|To experience Head you really need to understa...|positive|

そこで各レビュー文を入力として、その評価をRNNを用いて予測してみましょう。

**なおレビュー文(X)は、各単語を出現頻度順での数字に置き換えたものとして表され、評価(y)はpositiveを1、negativeを0として表しています。**

### 1. データセットの読み込み

`keras.datasets`内の`imdb.load_data`関数を用いてIMDbのデータセットを読み込みましょう。

```python
imdb.load_data(path="imdb.npz", num_words=None, skip_top=0, maxlen=None, seed=113, start_char=1, oov_char=2, index_from=3)
```
引数に関して簡単に紹介しておくと、
- **num_words**：最大インデックス（出現頻度上位`num_words-index_from`個の単語にのみIDが割り振られる）
- skip_top：各系列の冒頭読み飛ばし長
- maxlen：各系列の最大長（超過分は切り捨て）
- seed：系列のシャッフル用の乱数シード値
- **start_char**：系列開始（BOS）記号用インデックス
- **oov_char**：その他用（`skip_top`部分や`num_words`超過分）インデックス
- **index_from**：単語IDの開始インデックス

なお、0は通常後述するpaddingに用いられるため、意図的に回避したナンバリングがデフォルトとして設定されています。

読み込みの細かい設定については公式ドキュメントを参照してください。

参考：https://keras.io/ja/datasets/#imdb

In [10]:
pad_index = 0
num_words = 10000
(x_train, t_train), (x_test, t_test) = imdb.load_data(num_words=num_words)

x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.2, random_state=42)

# データセットサイズが大きいので演習用に短縮
x_train = x_train[:len(x_train)//2]
t_train = t_train[:len(t_train)//2]
x_valid = x_valid[:len(x_valid)//2]
t_valid = t_valid[:len(t_valid)//2]

17465344/17464789 [==============================] - 0s 0us/step


IMDbの各レビューは長さが異なります。したがって、可変長の系列に対してRNNを適用し、最後の隠れ状態を元に二値分類を行うということになります。

ですが、RNNの各バッチでの入力は同じ長さでないと、行列として計算が行えなくなってしまいます。

そこで実際にはまず、
- ミニバッチ内のデータの系列長を揃える（**padding**）

必要があります。

つまり系列の先頭あるいは末尾に、系列長の調整用であることを表す値（今回は0）を付加し、バッチ内の系列長を統一します。

これは `keras.preprocessing.sequence` にある関数 `pad_sequences` を使うことなどで可能です。

（ミニバッチごとの対応になるので、後ほど 4. 学習にて利用します）

参考：https://keras.io/ja/preprocessing/sequence/#pad_sequences

```python
x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index)) # バッチ毎のPadding
t_train_batch = np.array(t_train[start:end])[:, None]

_, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
```

またpaddingが多くなると計算的に非効率になるため、paddingを少なくする目的で予めデータの長さで降順にソートしておくことが多いです。

In [11]:
# trainデータを長さ順にソート
x_train_lens = [len(com) for com in x_train]
sorted_train_indexes = sorted(range(len(x_train_lens)), key=lambda x: -x_train_lens[x])

x_train = [x_train[ind] for ind in sorted_train_indexes]
t_train = [t_train[ind] for ind in sorted_train_indexes]

なお、paddingの部分はあくまで系列長を合わせるためなので、通常のRNNの計算はおこなわず、何らかの形で計算を無効にする必要があります。

その具体的な方法は後ほどRNN層の実装時に詳しく扱います。

### 2. 各層クラスの実装

#### 2.1. Embedding層

Embedding層では、単語を離散的なidから連続的な数百次元のベクトルに変換(埋め込み、embed)します。

下のEmbeddingクラスにおいて、入力$\boldsymbol{x}$は各行に文の単語のid列が入った行列で、重み$\boldsymbol{V}$は各行がそれぞれの単語idのベクトルに対応した行列です。

つまりそれぞれの行列のサイズは

- $\boldsymbol{x}$: (ミニバッチサイズ) x (ミニバッチ内の文の最大系列長)
- $\boldsymbol{V}$: (辞書の単語数) x (単語のベクトルの次元数)

です。

この$\boldsymbol{V}$から、入力$\boldsymbol{x}$のそれぞれの単語idに対して対応する単語ベクトルを取り出すことで、各単語をベクトルに変換します。

`tf`では`tf.nn.embedding_lookup`によりこの作業を行います。

この処理によって出力されるテンソルの次元数は、(ミニバッチサイズ) x (ミニバッチ内の文の最大系列長) x (単語のベクトルの次元数)となります。

![embedding](../figures/embedding.png)

$$m:\text{emb_dim}, \ n : \text{vocab_size}$$

参考：https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup

In [12]:
class Embedding:
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        self.V = tf.Variable(tf.random_normal([vocab_size, emb_dim], stddev=scale), name='V')

    def __call__(self, x):
        return tf.nn.embedding_lookup(self.V, x)

#### 2.2. RNN

RNNクラスでは、Embedding層で各単語がベクトルに変換されたものを入力として処理を行います。ここで入力$\boldsymbol{x}$は

- $\boldsymbol{x}$: (ミニバッチサイズ) x (ミニバッチ内の文の最大系列長) x (単語のベクトルの次元数)

となっています。`tf.scan`では第0軸方向に走査していくので、文の系列方向に沿って走査するために上の第0軸と第1軸を`tf.transpose`により入れ替えて

- $\boldsymbol{x}$: (ミニバッチ内の文の最大系列長) x (ミニバッチサイズ) x (単語のベクトルの次元数)

とします。

また先述の通り、**paddingの部分の計算を無効化**する必要があります。

ここではわかりやすい実装として、padding部では隠れ状態を変更しない、つまり前のステップの隠れ状態をそのままコピーすることにしましょう。

こうすることで、実際の系列の末尾における隠れ状態を保持するようにします。

実装としては、各系列の実際の系列長を表す`seq_len`を元に、実際に単語がある部分に1、padding部に0を置くバイナリマスク

$$\boldsymbol{m}=[m_1, m_2, \dots, m_t, \dots, m_T]$$

を`tf.sequence_mask`によって作成し、隠れ状態の更新時に以下のようにして適用します。（単純化のため文末にのみpaddingがあると想定します。）

$$
    \boldsymbol{h}_t = m_t \cdot \sigma\left(\boldsymbol{W}\left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}\right) + (1-m_t) \cdot \boldsymbol{h}_{t-1}
$$

こうすることでpaddingの部分では$\boldsymbol{h}_t=\boldsymbol{h}_{t-1}$となり、paddingの計算結果に対する影響がなくなります。

参考：https://www.tensorflow.org/api_docs/python/tf/sequence_mask

In [13]:
class RNN:
    def __init__(self, in_dim, hid_dim, seq_len=None, scale=0.08):
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        
        glorot = tf.cast(tf.sqrt(6/(in_dim + hid_dim*2)), tf.float32)
        self.W = tf.Variable(tf.random_uniform([in_dim+hid_dim, hid_dim], minval=-glorot, maxval=glorot), name='W')
        self.b = tf.Variable(tf.zeros([hid_dim]), name='b')
        
        self.seq_len = seq_len
        self.initial_state = None

    def __call__(self, x):
        def fn(h_prev, x_and_m):
            x_t, m_t = x_and_m
            inputs = tf.concat([x_t, h_prev], -1)
            # RNN
            h_t = tf.nn.tanh(tf.matmul(inputs, self.W) + self.b)
            # マスクの適用
            h_t = m_t * h_t + (1 - m_t) * h_prev
          
            return h_t

        # 入力の時間順化
        # shape: [batch_size, max_seqence_length, in_dim] -> [max_seqence_length, batch_size, in_dim]
        x_tmaj = tf.transpose(x, perm=[1, 0, 2])
        
        # マスクの生成＆時間順化
        mask = tf.cast(tf.sequence_mask(self.seq_len, tf.shape(x)[1]), tf.float32)
        mask_tmaj = tf.transpose(tf.expand_dims(mask, axis=-1), perm=[1, 0, 2])
        
        if self.initial_state is None:
            batch_size = tf.shape(x)[0]
            self.initial_state = tf.zeros([batch_size, self.hid_dim])
        
        h = tf.scan(fn=fn, elems=[x_tmaj, mask_tmaj], initializer=self.initial_state)
        
        return h[-1]

### 3. 計算グラフ構築 & パラメータの更新設定

In [14]:
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

In [15]:
tf.reset_default_graph() # グラフ初期化

emb_dim = 100
hid_dim = 50

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.float32, [None, None], name='t')

seq_len = tf.reduce_sum(tf.cast(tf.not_equal(x, pad_index), tf.int32), axis=1)

h = Embedding(num_words, emb_dim)(x)
h = RNN(emb_dim, hid_dim, seq_len)(h)
y = tf.layers.Dense(1, tf.nn.sigmoid)(h)

cost = -tf.reduce_mean(t*tf_log(y) + (1 - t)*tf_log(1 - y))

train = tf.train.AdamOptimizer().minimize(cost)
test = tf.round(y)

### 4. 学習

バッチの入力に注目しつつ、学習ループを見てみましょう。

In [16]:
n_epochs = 5
batch_size = 100
n_batches_train = len(x_train) // batch_size
n_batches_valid = len(x_valid) // batch_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        # Train
        train_costs = []
        for i in range(n_batches_train):
            start = i * batch_size
            end = start + batch_size
            
            x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index)) # バッチ毎のPadding
            t_train_batch = np.array(t_train[start:end])[:, None]

            _, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
            train_costs.append(train_cost)
        
        # Valid
        valid_costs = []
        y_pred = []
        for i in range(n_batches_valid):
            start = i * batch_size
            end = start + batch_size
            
            x_valid_pad = np.array(pad_sequences(x_valid[start:end], padding='post', value=pad_index)) # バッチ毎のPadding
            t_valid_pad = np.array(t_valid[start:end])[:, None]
            
            pred, valid_cost = sess.run([test, cost], feed_dict={x: x_valid_pad, t: t_valid_pad})
            y_pred += pred.flatten().tolist()
            valid_costs.append(valid_cost)
        print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f, Validation F1: %.3f' % (epoch+1, np.mean(train_costs), np.mean(valid_costs), f1_score(t_valid, y_pred, average='macro')))

EPOCH: 1, Training Cost: 0.600, Validation Cost: 0.460, Validation F1: 0.791
EPOCH: 2, Training Cost: 0.384, Validation Cost: 0.462, Validation F1: 0.798
EPOCH: 3, Training Cost: 0.294, Validation Cost: 0.484, Validation F1: 0.809
EPOCH: 4, Training Cost: 0.232, Validation Cost: 0.602, Validation F1: 0.698
EPOCH: 5, Training Cost: 0.190, Validation Cost: 0.611, Validation F1: 0.729


## 課題3. Cellを用いたRNNの記述

課題2. までは原理の理解のため、loopを構成する方法としてscanを紹介・利用の上、RNNを実装しました。

ここからはより実践的な実装として、RNNの各時点の処理に対応する**Cell構造**を用いたRNNの実装を扱います。

この方法では明示的にloopを構成することなくRNNを実装できます。

具体的には、`tf.nn.rnn_cell.BasicRNNCell()`を用いてCell構造を生成した後、`tf.nn.static_rnn`でCellに基づいたRNNを構成します。

参考：

- https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicRNNCell

- https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn

In [17]:
class RNN:
    def __init__(self, hid_dim, initial_state, maxlen):
        self.cell = tf.nn.rnn_cell.BasicRNNCell(hid_dim)
        self.initial_state = initial_state
        self.maxlen = maxlen

    def __call__(self, x):
        inputs = tf.unstack(x, num=self.maxlen, axis=1)
        outputs, state = tf.nn.static_rnn(self.cell, inputs, self.initial_state)
        return outputs[-1]
    
    # あるいは以下のようにも書ける
    #def __call__(self, x):
    #    outputs = []
    #    state = self.initial_state
    #    with tf.variable_scope("RNN"):
    #        for t in range(self.maxlen):
    #            if t > 0:
    #                tf.get_variable_scope().reuse_variables()
    #            (cell_output, state) = self.cell(x[:, t, :], state)
    #            outputs.append(cell_output)
    #    return outputs[-1]

しかし、このような処理では可変長の系列を適切に扱うことができません。

具体的には

- maxlenを明示的かつ固定長で与える必要があり、ミニバッチ毎に異なるmaxlenへ対応できない
- バッチ内でも系列ごとに長さは異なり、マスキングが必要

となってしまっています。

2点目のマスキングについては、実は`tf.nn.static_rnn`に引数`sequence_length`としてバッチ内の各系列長を与えれば解決が可能です。

1点目については、`tf.nn.static_rnn`に代えて`tf.nn.dynamic_rnn`という関数を用いることで、ミニバッチ毎にmaxlenが異なっても対応可能になります。

またこちらも引数`sequence_length`を受け取るため、2つの難点を同時に解決できます。よって以降基本的に`tf.nn.dynamic_rnn`を用いることとします。

（なお後述するeager modeを使用しても解決が可能です）

`static_rnn`ではテンソルの**リスト**（長さ：maxlen）で`input`を受け取っていましたが、`dynamic_rnn`では`[batch_size, maxlen, emb_dim]`のテンソルで受け取っているので注意してください。

参考：https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn

In [18]:
class RNN:
    def __init__(self, hid_dim, seq_len = None, initial_state = None):
        self.cell = tf.nn.rnn_cell.BasicRNNCell(hid_dim)
        self.initial_state = initial_state
        self.seq_len = seq_len
    
    def __call__(self, x):
        if self.initial_state is None:
            self.initial_state = self.cell.zero_state(tf.shape(x)[0], tf.float32)
            
        # outputsは各系列長分以降は0になるので注意
        outputs, state = tf.nn.dynamic_rnn(self.cell, x, self.seq_len, self.initial_state)
        return tf.gather_nd(outputs, tf.stack([tf.range(tf.shape(x)[0]), self.seq_len-1], axis=1))

In [19]:
tf.reset_default_graph() # グラフ初期化

emb_dim = 100
hid_dim = 50

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.float32, [None, None], name='t')

seq_len = tf.reduce_sum(tf.cast(tf.not_equal(x, pad_index), tf.int32), axis=1)

h = Embedding(num_words, emb_dim)(x)
h = RNN(hid_dim, seq_len)(h)
y = tf.layers.Dense(1, tf.nn.sigmoid)(h)

cost = -tf.reduce_mean(t*tf_log(y) + (1 - t)*tf_log(1 - y))

train = tf.train.AdamOptimizer().minimize(cost)
test = tf.round(y)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [20]:
n_epochs = 5
batch_size = 100
n_batches_train = len(x_train) // batch_size
n_batches_valid = len(x_valid) // batch_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        # Train
        train_costs = []
        for i in range(n_batches_train):
            start = i * batch_size
            end = start + batch_size
            
            x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index))
            t_train_batch = np.array(t_train[start:end])[:, None]

            _, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
            train_costs.append(train_cost)
        
        # Valid
        valid_costs = []
        y_pred = []
        for i in range(n_batches_valid):
            start = i * batch_size
            end = start + batch_size
            
            x_valid_pad = np.array(pad_sequences(x_valid[start:end], padding='post', value=pad_index))
            t_valid_pad = np.array(t_valid[start:end])[:, None]
            
            pred, valid_cost = sess.run([test, cost], feed_dict={x: x_valid_pad, t: t_valid_pad})
            y_pred += pred.flatten().tolist()
            valid_costs.append(valid_cost)
        print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f, Validation F1: %.3f' % (epoch+1, np.mean(train_costs), np.mean(valid_costs), f1_score(t_valid, y_pred, average='macro')))

EPOCH: 1, Training Cost: 0.624, Validation Cost: 0.480, Validation F1: 0.782
EPOCH: 2, Training Cost: 0.505, Validation Cost: 0.455, Validation F1: 0.792
EPOCH: 3, Training Cost: 0.450, Validation Cost: 0.539, Validation F1: 0.724
EPOCH: 4, Training Cost: 0.257, Validation Cost: 0.483, Validation F1: 0.787
EPOCH: 5, Training Cost: 0.156, Validation Cost: 0.571, Validation F1: 0.784


## 課題4. Long short-term memory (LSTM)

実装する式は次のようになります。($\odot$は要素ごとの積)

- 入力ゲート: $\hspace{20mm}\boldsymbol{i}_t = \mathrm{\sigma} \left(\boldsymbol{W}_i \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_i\right)$
- 忘却ゲート: $\hspace{20mm}\boldsymbol{f}_t = \mathrm{\sigma} \left(\boldsymbol{W}_f \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_f\right)$  
- 出力ゲート: $\hspace{20mm}\boldsymbol{o}_t = \mathrm{\sigma} \left(\boldsymbol{W}_o \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_o\right)$  
- セル:　　　 $\hspace{20mm}\boldsymbol{c}_t = \boldsymbol{f}_t \odot \boldsymbol{c}_{t-1} + \boldsymbol{i}_t \odot \tanh \left(\boldsymbol{W}_c \left[\begin{array}{c} \boldsymbol{x}_t \\ \boldsymbol{h}_{t-1} \end{array}\right] + \boldsymbol{b}_c\right)$
- 隠れ状態: 　$\hspace{20mm}\boldsymbol{h}_t = \boldsymbol{o}_t \odot \tanh \left(\boldsymbol{c}_t \right)$

単純なRNNでは各ステップの関数の戻り値は隠れ状態のみ ($\boldsymbol{h}_t$) でしたが、LSTMではセル状態と隠れ状態の2つ ($\boldsymbol{c}_t, \boldsymbol{h}_t$) となるので注意してください。

またマスクに関しても両方に適用する必要があります。

まずは、愚直に`tf.scan`を用いて実装してみましょう。

In [21]:
class LSTM:
    def __init__(self, in_dim, hid_dim, seq_len = None, initial_state = None):
        self.in_dim = in_dim
        self.hid_dim = hid_dim

        glorot = tf.cast(tf.sqrt(6/(in_dim + hid_dim*2)), tf.float32)
        
        # 入力ゲート
        self.W_i = tf.Variable(tf.random_uniform([in_dim + hid_dim, hid_dim], minval=-glorot, maxval=glorot), name='W_i')
        self.b_i  = tf.Variable(tf.zeros([hid_dim]), name='b_i')
        
        # 忘却ゲート
        self.W_f = tf.Variable(tf.random_uniform([in_dim + hid_dim, hid_dim], minval=-glorot, maxval=glorot), name='W_f')
        self.b_f  = tf.Variable(tf.zeros([hid_dim]), name='b_f')

        # 出力ゲート
        self.W_o = tf.Variable(tf.random_uniform([in_dim + hid_dim, hid_dim], minval=-glorot, maxval=glorot), name='W_o')
        self.b_o  = tf.Variable(tf.zeros([hid_dim]), name='b_o')

        # セル
        self.W_c = tf.Variable(tf.random_uniform([in_dim + hid_dim, hid_dim], minval=-glorot, maxval=glorot), name='W_c')
        self.b_c  = tf.Variable(tf.zeros([hid_dim]), name='b_c')

        # マスク
        self.seq_len = seq_len
        
        self.initial_state = initial_state

    def __call__(self, x):
        def fn(prev_state, x_and_m):
            c_prev, h_prev = prev_state[0], prev_state[1]
            x_t, m_t = x_and_m
            
            inputs = tf.concat([x_t, h_prev], -1)
            
            # 各ゲート
            i_t = tf.nn.sigmoid(tf.matmul(inputs, self.W_i) + self.b_i)
            f_t = tf.nn.sigmoid(tf.matmul(inputs, self.W_f) + self.b_f)
            o_t = tf.nn.sigmoid(tf.matmul(inputs, self.W_o) + self.b_o)

            # セル
            c_t = f_t * c_prev + i_t * tf.nn.tanh(tf.matmul(inputs, self.W_c) + self.b_c)

            # 隠れ状態
            h_t = o_t * tf.nn.tanh(c_t)
            
            # マスクの適用
            c_t = m_t * c_t + (1 - m_t) * c_prev
            h_t = m_t * h_t + (1 - m_t) * h_prev

            return tf.stack([c_t, h_t])

        # 入力の時間順化
        x_tmaj = tf.transpose(x, perm=[1, 0, 2])
        
        # マスクの生成＆時間順化
        mask = tf.cast(tf.sequence_mask(self.seq_len, tf.shape(x)[1]), tf.float32)
        mask_tmaj = tf.transpose(tf.expand_dims(mask, axis=-1), perm=[1, 0, 2])
        
        if self.initial_state is None:
            batch_size = tf.shape(x)[0]
            self.initial_state = tf.stack([tf.zeros([batch_size, self.hid_dim]), tf.zeros([batch_size, self.hid_dim])])

        state_seq = tf.scan(fn=fn, elems=[x_tmaj, mask_tmaj], initializer=self.initial_state)
        
        return state_seq[-1][1]

RNNとLSTMで同じタスクで学習させてみて、比較してみましょう。

In [22]:
tf.reset_default_graph() # グラフ初期化

emb_dim = 100
hid_dim = 50

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.float32, [None, None], name='t')

seq_len = tf.reduce_sum(tf.cast(tf.not_equal(x, pad_index), tf.int32), axis=1)

h = Embedding(num_words, emb_dim)(x)
h = LSTM(emb_dim, hid_dim, seq_len)(h)
y = tf.layers.Dense(1, tf.nn.sigmoid)(h)

cost = -tf.reduce_mean(t*tf_log(y) + (1 - t)*tf_log(1 - y))

train = tf.train.AdamOptimizer().minimize(cost)
test = tf.round(y)

In [23]:
n_epochs = 5
batch_size = 100
n_batches_train = len(x_train) // batch_size
n_batches_valid = len(x_valid) // batch_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        # Train
        train_costs = []
        for i in range(n_batches_train):
            start = i * batch_size
            end = start + batch_size
            
            x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index))
            t_train_batch = np.array(t_train[start:end])[:, None]

            _, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
            train_costs.append(train_cost)
        
        # Valid
        valid_costs = []
        y_pred = []
        for i in range(n_batches_valid):
            start = i * batch_size
            end = start + batch_size
            
            x_valid_pad = np.array(pad_sequences(x_valid[start:end], padding='post', value=pad_index))
            t_valid_pad = np.array(t_valid[start:end])[:, None]
            
            pred, valid_cost = sess.run([test, cost], feed_dict={x: x_valid_pad, t: t_valid_pad})
            y_pred += pred.flatten().tolist()
            valid_costs.append(valid_cost)
        print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f, Validation F1: %.3f' % (epoch+1, np.mean(train_costs), np.mean(valid_costs), f1_score(t_valid, y_pred, average='macro')))

EPOCH: 1, Training Cost: 0.617, Validation Cost: 0.467, Validation F1: 0.789
EPOCH: 2, Training Cost: 0.348, Validation Cost: 0.393, Validation F1: 0.838
EPOCH: 3, Training Cost: 0.237, Validation Cost: 0.432, Validation F1: 0.843
EPOCH: 4, Training Cost: 0.241, Validation Cost: 0.452, Validation F1: 0.822
EPOCH: 5, Training Cost: 0.145, Validation Cost: 0.510, Validation F1: 0.826


LSTMについても、以下のようにしてcellを用いることができます。`tf.nn.rnn_cell.BasicLSTMCell`を使用しましょう。

In [24]:
class LSTM:
    def __init__(self, hid_dim, seq_len = None, initial_state = None):
        self.cell = tf.nn.rnn_cell.BasicLSTMCell(hid_dim)
        self.initial_state = initial_state
        self.seq_len = seq_len

    def __call__(self, x):
        if self.initial_state is None:
            self.initial_state = self.cell.zero_state(tf.shape(x)[0], tf.float32)
            
        outputs, state = tf.nn.dynamic_rnn(self.cell, x, self.seq_len, self.initial_state)
        return tf.gather_nd(outputs, tf.stack([tf.range(tf.shape(x)[0]), self.seq_len-1], axis=1))

## 【補足】Gradient Clipping（長系列への対処法）

LSTMは長系列に対しても学習がうまく行きやすいモデルでしたが、一般のRNNにおける長系列の学習のTipsとして、**Gradient Clipping**に触れておきます。

RNNでは誤差逆伝播法が特に**Back Propagation Through Time (BPTT)**と呼ばれるものになり、各層のみならず各時点の勾配が乗算されます。

そのため、通常よりも勾配が過大（或いは過小）になりやすいという特徴をもっています。

こうした現象を**勾配爆発（消失）**と呼びますが、勾配爆発は学習を不安定化し収束を困難にします。

![Clipping](../figures/Clipping.png)
出典：Ian Goodfellow et. al, “Deep Learning”, MIT press, 2016 (http://www.deeplearningbook.org/)

そこで、勾配の大きさを意図的に制限して対処しようというのが、Gradient Clippingと呼ばれる手法です。

以下のように、明示的にoptimizerから勾配を取得した後、`tf.clip_by_value`関数に通した上で勾配を適用することで実行できます。

In [25]:
# train = tf.train.AdamOptimizer().minimize(cost) を以下に置き換え

optimizer = tf.train.AdamOptimizer()
grads = optimizer.compute_gradients(cost)
clipped_grads = [(tf.clip_by_value(grad_val, -1., 1.), var) for grad_val, var in grads]
train = optimizer.apply_gradients(clipped_grads)

## 【補足】 Eager Executionについて

近年、Define-and-RunであったTensorFlowに、Define-by-Runを可能にするEager Executionが導入されました。

Eager Executionを用いることでTensorFlowでも動的な計算グラフの構築が実現され、例えばミニバッチ毎に処理を変えるといったことが可能になりました。

もちろん、Define-by-Runには動的である故のデメリット（ex. 最適化が困難など）もあります。

しかし、上述のメリットにより、特にRNN系のモデルの記述には重宝されることが多いため、今回補足事項として簡単に取り扱っておきたいと思います。

In [26]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# eager executionでレイヤーを定義するには、tf.keras.layers.Layerを継承する必要があります
class EagerEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        super(EagerEmbedding, self).__init__()
        
        # self.add_variableでvariableの追加を行います
        self.V = self.add_variable("V", [vocab_size, emb_dim], initializer='RandomNormal')

    # call関数に順伝播を実装します
    def call(self, inputs):
        return tf.nn.embedding_lookup(self.V, inputs)

class EagerRNN(tf.keras.layers.Layer):
    def __init__(self, hid_dim):
        super(EagerRNN, self).__init__()
        
        self.hid_dim = hid_dim
    
    # build関数でもvariableの追加が可能です（特にinput_shapeに依存したvariable）
    def build(self, input_shape):
        self.W = self.add_variable("W", [input_shape[-1] + self.hid_dim, self.hid_dim], initializer='Orthogonal')
        self.b = self.add_variable("b", [self.hid_dim], initializer='Zeros')

    # ここではマスクを考慮せずに書いています、意欲的な方はぜひマスクを考慮したrnnに書き換えてみてください
    def call(self, inputs):
        outputs = []
        state = tf.zeros(shape=(inputs.shape[0], self.hid_dim))
        for t in range(inputs.shape[1]):
            state = tf.nn.tanh(tf.matmul(tf.concat([inputs[:,t,:], state], axis=1), self.W) + self.b)
            output = state
            outputs.append(output)
        return outputs[-1]

# なお、RNNをcellを用いてeager executionで実装することも可能です（こちらについてもぜひ挑戦してみてください）

# モデルを一つにまとめるには、tf.keras.Modelを継承したクラスを用います（あるいはtf.keras.Sequential）
class Model(tf.keras.Model):
    def __init__(self, vocab_size, emb_dim, hid_dim, scale=0.08):
        super(Model, self).__init__()
        
        self.word_embedding = EagerEmbedding(vocab_size, emb_dim)
        self.rnn = EagerRNN(hid_dim)
        self.dense = tf.keras.layers.Dense(1, activation=None)

    def call(self, inputs):
        h = self.word_embedding(inputs)
        h = self.rnn(h)
        y = self.dense(h)
        return tf.reshape(y, shape=[-1])
    
# loss関数
def loss_with_logits(logits, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf.cast(labels, tf.float32)))

In [27]:
# eager executionでの実行を宣言(tensorflowの関数実行の最小に行う必要がある、要kernel restart)
tf.enable_eager_execution()

ValueError: tf.enable_eager_execution must be called at program startup.

In [28]:
from keras.datasets import imdb
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

pad_index = 0
num_words = 10000
(x_train, t_train), (x_test, t_test) = imdb.load_data(num_words=num_words)
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.2, random_state=42)

x_train = x_train[:len(x_train)//2]
t_train = t_train[:len(t_train)//2]
x_valid = x_valid[:len(x_valid)//2]
t_valid = t_valid[:len(t_valid)//2]

x_train = pad_sequences(x_train, padding='post', value=pad_index)
x_valid = pad_sequences(x_valid, padding='post', value=pad_index)

batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, t_train))
train_dataset = train_dataset.shuffle(1000).batch(batch_size)

ImportError: No module named 'keras'

In [29]:
emb_dim = 100
hid_dim = 50
model = Model(num_words, emb_dim, hid_dim)

optimizer = tf.train.AdamOptimizer()

epoch_num = 10

# eager executionはデフォルトでcpu実行なので、gpuを指定
with tf.device("/gpu:0"):
    for epoch in range(epoch_num):
        tf.train.get_or_create_global_step()

        for (i, (inputs, labels)) in enumerate(train_dataset):
            # eager executionではminimize関数にはlossを関数として渡す必要があります（引数無しlambda）
            optimizer.minimize(lambda: loss_with_logits(model(inputs), labels), global_step=tf.train.get_global_step())

        valid_logits = model(x_valid)
        valid_loss = loss_with_logits(valid_logits, t_valid)
        valid_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.nn.sigmoid(valid_logits)), t_valid), tf.float32))

        print(('EPOCH %02d\t Validation Loss: %.2f\t Validation Accuracy: %.2f') % (epoch + 1, valid_loss, valid_accuracy))

NameError: name 'train_dataset' is not defined

実装において、kerasのクラスを継承していたことからも想像できるように、kerasの各種の便利な機能を用いるにあたっても比較的相性が良くなっています。

たとえば、上記の学習ループはkerasの`Model.fit()`などに置き換えることが可能です。興味のある方は調べてみてください。